In [1]:
! pip install yfinance

In [2]:
import numpy as np
from sklearn.metrics import accuracy_score
import numba
import pandas as pd
import random
from datetime import datetime, timezone, timedelta
from enum import Enum
from typing import List, Tuple, Dict, Any
import matplotlib.pyplot as plt
import pytz
import requests
from DataImportfromYf import DataImportfromYf
from Classes import Frequency, Position, Config, PositionType, Weight, BaseWeightComputation
from model_validation import get_train_val_test_idx_rolling, get_train_val_test_idx_regular
from Features import add_return, binarize_label, standardize_feat_basket
from keras.layers import LSTM, Dense, Dropout, Input, BatchNormalization 
from keras.models import load_model, model_from_json, Sequential, Model
from keras.callbacks import EarlyStopping
from keras.utils import plot_model
from keras.optimizers import Adam
from keras.initializers import RandomUniform, GlorotUniform, GlorotNormal
from LSTM_model import LongShortTermMemory
from Backtest_classes import PositionGenerator, Backtester


# Data Loading 
Define the universe, the time range and interval considered (2002-05-01 To 2020-01-01)

In [4]:
universe = ["GETI-B.ST", "ATCO-A.ST", "VOLV-B.ST", "TEL2-B.ST", "ELUX-B.ST", "TELIA.ST", "HM-B.ST", "SKA-B.ST", 
            "ALFA.ST", "ERIC-B.ST", "SKF-B.ST", "SAND.ST", "AZN.ST", "SECU-B.ST", "INVE-B.ST", "ABB.ST", 
            "SEB-A.ST", "SHB-A.ST", "SCA-B.ST", "SSAB-A.ST", "ASSA-B.ST", "ALIV-SDB.ST", "HEXA-B.ST", 
            "SWED-A.ST", "ATCO-B.ST", "KINV-B.ST", "BOL.ST", "NDA-SE.ST"]
start_date = datetime(2002, 5, 1)
end_date = datetime(2020, 1, 1)
interval = "1d"

obj = DataImportfromYf(universe=universe, start_ts=start_date, end_ts=end_date, interval=interval, ignore_tz=True)
data = obj.get_data()

[*********************100%***********************]  28 of 28 completed


In [5]:
data.head(5)

adj_close       close        high         low  \
ts         symbol                                                        
2002-05-01 ABB.ST        68.324242   73.443237   73.443237   73.443237   
           ALIV-SDB.ST  190.604477  228.500000  228.500000  228.500000   
           ATCO-A.ST      3.608177    7.640396    7.640396    7.640396   
           ATCO-B.ST      3.110864    7.141966    7.141966    7.141966   
           AZN.ST       402.793365  482.000000  482.000000  482.000000   

                              open  volume  
ts         symbol                           
2002-05-01 ABB.ST        73.443237     0.0  
           ALIV-SDB.ST  228.500000     0.0  
           ATCO-A.ST      7.640396     0.0  
           ATCO-B.ST      7.141966     0.0  
           AZN.ST       482.000000     0.0

## Data cleaning

In [6]:
# Are they any missing values ?
data[data.isnull().any(axis=1)]

,,adj_close,close,high,low,open,volume
ts,symbol,,,,,,


## Add past, futur returns and target

In [7]:
# Past Returns
data = add_return(data, target="close", deltas=[1], by="symbol",disable_tqdm=True)
# Futur Returns
data = add_return(data, target="close", deltas=[-1], by="symbol",disable_tqdm=True)

In [8]:
# Drop useless data
data = data.dropna(subset=["past_ret_1D_close","fut_ret_1D_close"])

In [9]:
# Target feature (y_label)
label_name = "fut_ret_1D_close"
label_fit = data[label_name]
label_fit = binarize_label(data=label_fit)

/home/ibenchek/projet/projet-ZZ2/LSTM/Features.py:75: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  data = data.groupby("ts").apply(lambda df: pd.Series((df > df.median()).astype(int), index=df.index))


In [10]:
data = standardize_feat_basket(data=data, features_names=["past_ret_1D_close"], by=["ts"])

In [11]:
len(data[data.isnull().any(axis=1)])

891

In [12]:
# Dropna introtuced by the standardization process
data = data.dropna(subset=["past_ret_1D_close","fut_ret_1D_close"])

# Statstical Learning 

## Train and save all LSTM models for each stock

In [19]:
def train_all_lstm(data:pd.DataFrame, label_fit: pd.Series, feature: str, symbol_list: List[str],
                    strategy_code: str, model_parameters: Dict[str, Any], path_model: str) -> None:
    for symbol in symbol_list:
        lstm = LongShortTermMemory(data, label_fit, feature, symbol, strategy_code, model_parameters, path_model)
        # Split the ts in three different sets
        train_idx, val_idx, test_idx = lstm.split_series()
        # Return all datasets (x_train, x_val, x_test, etc.) in a dict
        res = lstm.get_data(train_idx, val_idx, test_idx)
        # Shape the data as recomanded in the keras api
        shaped_data = lstm.shape_data(res)
        # Fit the LSTM model
        lstm.fit(shaped_data)
        # Save the model
        lstm.save_lstm_model(path_model)

In [20]:
# config variables
path_model = f"./Models/"
opt = Adam(learning_rate=0.001)
model_parameters = {"split_ratio" : {"train" : 0.70, "val" : 0.15}, "n_steps" : 240,
                    "early_stopp_patience" : 5, "epochs" : 100, "batch_size": 30, "init" : 0, 
                    "n_neurons" : 3, "dropout" : 0.06, "recurrent_dropout" : 0, "metrics" : "accuracy", 
                    "optimizer" : opt, "kernel_init" : GlorotUniform(seed=123),
                    "path_model" : path_model}

In [21]:
train_all_lstm(data, label_fit, "past_ret_1D_close", universe, "my_first_strat", model_parameters, path_model)

Epoch 1/100
96/96 - 12s - loss: 1.4471 - accuracy: 0.4983 - val_loss: 0.7804 - val_accuracy: 0.5330 - 12s/epoch - 123ms/step
Epoch 2/100
96/96 - 8s - loss: 0.8397 - accuracy: 0.4885 - val_loss: 0.7155 - val_accuracy: 0.5165 - 8s/epoch - 79ms/step
Epoch 3/100
96/96 - 7s - loss: 0.7896 - accuracy: 0.4783 - val_loss: 0.7099 - val_accuracy: 0.5094 - 7s/epoch - 76ms/step
Epoch 4/100
96/96 - 8s - loss: 0.7586 - accuracy: 0.4871 - val_loss: 0.7073 - val_accuracy: 0.5189 - 8s/epoch - 81ms/step
Epoch 5/100
96/96 - 7s - loss: 0.7458 - accuracy: 0.4888 - val_loss: 0.7037 - val_accuracy: 0.5189 - 7s/epoch - 73ms/step
Epoch 6/100
96/96 - 8s - loss: 0.7279 - accuracy: 0.4930 - val_loss: 0.7024 - val_accuracy: 0.5165 - 8s/epoch - 79ms/step
Epoch 7/100
96/96 - 7s - loss: 0.7229 - accuracy: 0.4864 - val_loss: 0.7023 - val_accuracy: 0.4976 - 7s/epoch - 74ms/step
Epoch 8/100
96/96 - 8s - loss: 0.7377 - accuracy: 0.4860 - val_loss: 0.7006 - val_accuracy: 0.5094 - 8s/epoch - 80ms/step
Epoch 9/100
96/96 - 7